In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install -U pysal

     |████████████████████████████████| 332.1MB 58kB/s 
     |████████████████████████████████| 3.0MB 50.8MB/s 
     |████████████████████████████████| 419kB 46.9MB/s 
     |████████████████████████████████| 61kB 25.4MB/s 
     |████████████████████████████████| 25.0MB 3.4MB/s 
     |████████████████████████████████| 81kB 29.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/ca/b9/69/49a64348ddb1feaf76b426c81c1db7275cfabacd8bf4ccd778
Successfully built pysal


In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import scipy
import matplotlib.pylab as plt
import scipy.sparse as sparse

In [6]:
def load_dataset(img_rows, img_cols):
	from tensorflow.keras.datasets import mnist
	num_classes = 10

	img_rows, img_cols = 28, 28

	(x_train, y_train), (x_test, y_test) = mnist.load_data()

	x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols, 1)
	x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols, 1)
	input_shape = (img_rows, img_cols, 1)

	x_train = x_train.astype('float32')
	x_test = x_test.astype('float32')
	x_train /= 255
	x_test /= 255
	print('x_train shape:', x_train.shape)
	print(x_train.shape[0], 'train samples')
	print(x_test.shape[0], 'test samples')

	y_train = keras.utils.to_categorical(y_train, num_classes)
	y_test = keras.utils.to_categorical(y_test, num_classes)

	return (x_train, y_train), (x_test, y_test)


# make fully connected adjacency matrix
def make_full_adjacency(size):
	import pysal

	A = pysal.lib.weights.lat2W(size, size, rook=False, id_type="int")
	A, _ = A.full()
	A = np.array(A)
	A = A.astype(np.int32)

	return A


def calculate_degree_matrix(A):

	in_degree = np.sum(A, axis=0)
	out_degree = np.sum(A, axis=1)

	diag = in_degree + out_degree

	D = np.diag(diag) - np.eye(diag.size)

	return D


def reduce_dataset(train, validation, batch_size):
	(x_train, y_train) = train
	(x_test, y_test) = validation

	extra_elements = x_train.shape[0] % batch_size
	if extra_elements:
		x_train = x_train[0:x_train.shape[0] - extra_elements]
		y_train = y_train[0:y_train.shape[0] - extra_elements]

	extra_elements = x_test.shape[0] % batch_size
	if extra_elements:
		x_test = x_test[0:x_test.shape[0] - extra_elements]
		y_test = y_test[0:y_test.shape[0] - extra_elements]

	return (x_train, y_train), (x_test, y_test)

In [9]:
class GCN(keras.layers.Layer):

	def __init__(self, A_hat, F_prime, batch_size, **kwargs):
		self.A = self.convert_sparse_matrix_to_sparse_tensor(A_hat)
		self.F_prime = F_prime
		self.batch_size = batch_size
		self.size = self.A.shape[0]/self.batch_size

		super(GCN, self).__init__(**kwargs)

	def build(self, input_shape):
		self.W = self.add_weight(name='W',
	                              shape=(input_shape[2], self.F_prime),
	                              initializer='uniform',
	                              trainable=True)
		super(GCN, self).build(input_shape)

	def call(self, x):
		# x = tf.cast(x, tf.float64)
		x = tf.reshape(x, [-1, 1])

		# if x.get_shape()[0] and x.get_shape()[0] / self.size != self.batch_size:
		# 	A_reduced = self.A.copy()
		# 	A_reduced.resize(self.size * x.get_shape()[0], self.size * x.get_shape()[0])
		# 	H = tf.sparse.sparse_dense_matmul(self.A_reduced, x)
		# else:
        H = tf.sparse.sparse_dense_matmul(self.A, x)

		return tf.matmul(H, self.W)

	def compute_output_shape(self, input_shape):
		return (input_shape[0], self.F_prime)

	def convert_sparse_matrix_to_sparse_tensor(self, X):
		coo = X.tocoo()
		indices = np.mat([coo.row, coo.col]).transpose()
		return tf.SparseTensor(indices, coo.data, coo.shape)


class GCNPool(keras.layers.Layer):

	def __init__(self, shape, F_prime, batch_size, **kwargs):
		self.img_rows = shape[0]
		self.img_cols = shape[1]
		self.batch_size = batch_size

		self.F_prime = F_prime
		super(GCNPool, self).__init__(**kwargs)

	def call(self, x):
		segment_ids = np.array([], dtype=np.int32).reshape(0, self.img_rows * self.img_cols)
		for b in range(self.batch_size):
			index = np.repeat(b, self.img_rows * self.img_cols)
			segment_ids = np.concatenate((segment_ids, index), axis=None)

		return tf.math.segment_max(x, segment_ids)


	def compute_output_shape(self, input_shape):
		return (input_shape[0] / (self.img_rows * self.img_cols), self.F_prime)



TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-9-f45e3117ed86>, line 27)

In [10]:
def main():
	batch_size = 64
	epochs = 10
	img_rows, img_cols = 28, 28

	(x_train, y_train), (x_test, y_test) = load_dataset(img_rows, img_cols)
	(x_train, y_train), (x_test, y_test) = reduce_dataset((x_train, y_train), (x_test, y_test), batch_size)

	A = make_full_adjacency(img_rows)
	A_hat = A + np.eye(img_rows * img_cols)
	A_hat = A_hat.astype(np.float32)

	D_hat = calculate_degree_matrix(A_hat)

	D_hat = scipy.linalg.fractional_matrix_power(D_hat, -0.5)
	A_hat = np.linalg.multi_dot([D_hat, A_hat, D_hat])
	A_hat = sparse.csr_matrix(A_hat)

	batch_A_hat = sparse.kron(sparse.identity(batch_size), A_hat)
	batch_A_hat = batch_A_hat.astype(np.float32)
	# plt.subplot(221)
	# plt.spy(A_hat)
	# plt.subplot(222)
	# plt.spy(batch_A_hat)
	# plt.show()

	# plt.matshow(batch_A_hat.todense()[0:784*2, 0:784*2])
	plt.show()

	from tensorflow.keras.models import Sequential
	from tensorflow.keras.layers import Dense, ReLU

	model = Sequential()
	model.add(GCN(batch_A_hat, 32, batch_size, input_shape=(img_cols*img_rows, 1)))
	model.add(ReLU())
	model.add(GCNPool((img_rows, img_cols), 32, batch_size))
	model.add(Dense(10, activation='softmax'))

	model.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=keras.optimizers.Adadelta(),
	              metrics=['accuracy'])

	model.fit(x_train, y_train,
	          batch_size=batch_size,
	          epochs=epochs,
	          verbose=1,
	          validation_data=(x_test, y_test))
	score = model.evaluate(x_test, y_test,
	                       batch_size=batch_size,
	                       verbose=1)

	print('Test loss:', score[0])
	print('Test accuracy:', score[1])

	return

main()

x_train shape: (60000, 784, 1)
60000 train samples
10000 test samples
Train on 59968 samples, validate on 9984 samples
Epoch 1/10
19072/59968 [========>.....................] - ETA: 14s - loss: 2.3026 - accuracy: 0.0988

KeyboardInterrupt: 